In [42]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import pandas as pd
from collections import Counter
import re
import pickle
import MeCab
mc = MeCab.Tagger("-Ochasen")

# Excelのdata_frame化
- 現状開かれるファイル
    - A_W_171116
        - excelの行番号1～299行目までのデータを使用
    - A_W_171121
        - excelの行番号300～末尾行目までのデータを使用
    - S_H_171116
        - A_W_YYMMDDとは別の人たちが作成したexcelデータ
    - 斎藤_2015
        - S_H_1116の破損しているreply列のデータを修復するためのファイル
    

In [43]:
dirpath = "data/excel/"

filename = "A_W_171116.xlsx"
A_W_171116_excel = pd.ExcelFile(dirpath + filename)
sheet_name = A_W_171116_excel.sheet_names[0]
A_W_171116_df = A_W_171116_excel.parse(sheet_name)

filename = "A_W_171128.xlsx"
A_W_171128_excel = pd.ExcelFile(dirpath + filename)
sheet_name = A_W_171128_excel.sheet_names[0]
A_W_171128_df = A_W_171128_excel.parse(sheet_name)

filename = "171116.xlsx"
S_H_171116_excel = pd.ExcelFile(dirpath + filename)
sheet_name = S_H_171116_excel.sheet_names[0]
S_H_171116_df = S_H_171116_excel.parse(sheet_name)

filename = "2015斉藤.xlsx"
S_2015_excel = pd.ExcelFile(dirpath + filename)
sheet_name = S_2015_excel.sheet_names[0]
S_2015_df = S_2015_excel.parse(sheet_name)

In [44]:
A_W_171116_df.head(1)

No. 返信元  発言の起点  起点からの数                  時間  lid  gid  発言者ID ニックネーム  宛先者ID  \
0    1  \N      1       0 2015-06-16 16:05:00    1    1     52   まこぴす      0   

  ...  ?3 Social Watanabe.3 Unnamed: 29 ○×.3  ?4  Target Watanabe.4  \
0 ... NaN    NaN        NaN         NaN  1.0 NaN     NaN        NaN   

  Unnamed: 34  ?5  
0         NaN NaN  

[1 rows x 36 columns]

In [45]:
A_W_171128_df.head(1)

No. 返信元  発言の起点  起点からの数                  時間  lid  gid  発言者ID ニックネーム  宛先者ID  \
0  1.0  \N    1.0     0.0 2015-06-16 16:05:00  1.0  1.0   52.0   まこぴす    0.0   

  ...  ?3 Social(A) Watanabe.3 Unnamed: 29 ○×.3  ?4  Target(A) Watanabe.4  \
0 ... NaN       NaN        NaN           0  1.0 NaN        NaN        NaN   

  Unnamed: 34  ?5  
0         NaN NaN  

[1 rows x 36 columns]

In [46]:
S_H_171116_df.head(1)

id  date                  列1  reply  lid  gid   cname label         body  \
0  1593  1593 2015-07-21 06:19:00      2    1    2  世界のわたべ    全員  よろしくお願いします。   

  Epistemic(H)     ...     ?(3) Social(H)  Social(S) Social(FA) ?(5)  \
0     Off Task     ...      NaN       NaN        NaN        NaN  NaN   

  Social Target  ?(ST) Unnamed: 27 Unnamed: 28 Unnamed: 29  
0           NaN    NaN         NaN         NaN         NaN  

[1 rows x 30 columns]

In [47]:
S_2015_df.head(1)

発言番号 返信元  発言の起点  起点からの数                  時間  講義番号  グループ番号  発言者ID  ニックネーム  \
0  1593  \N   1593       0 2015-07-21 06:19:00     2       1      2  世界のわたべ   

   宛先者ID  宛先         発言内容  
0      0  全員  よろしくお願いします。

In [48]:
print(A_W_171116_df.shape)
print(A_W_171128_df.shape)
print(S_H_171116_df.shape)
print(S_2015_df.shape)

(4943, 36)
(4945, 36)
(5017, 30)
(5017, 12)


In [49]:
for i ,(columns1116, columns1128) in enumerate(zip(A_W_171116_df, A_W_171128_df)):
    print(i, columns1116, columns1128)

0 No. No.
1 返信元 返信元
2 発言の起点 発言の起点
3 起点からの数 起点からの数
4 時間 時間
5 lid lid
6 gid gid
7 発言者ID 発言者ID
8 ニックネーム ニックネーム
9 宛先者ID 宛先者ID
10 宛先 宛先
11 発言内容 発言内容
12 Epistemic Epistemic(A)
13 Watanabe Watanabe
14 Unnamed: 14 Unnamed: 14
15 ○× ○×
16 ? ?
17 Coordination Coordination(A)
18 Watanabe.1 Watanabe.1
19 Unnamed: 19 Unnamed: 19
20 ○×.1 ○×.1
21 ?2 ?2
22 Argumentation Argumentation(A)
23 Watanabe.2 Watanabe.2
24 Unnamed: 24 Unnamed: 24
25 ○×.2 ○×.2
26 ?3 ?3
27 Social Social(A)
28 Watanabe.3 Watanabe.3
29 Unnamed: 29 Unnamed: 29
30 ○×.3 ○×.3
31 ?4 ?4
32 Target Target(A)
33 Watanabe.4 Watanabe.4
34 Unnamed: 34 Unnamed: 34
35 ?5 ?5


In [50]:
for i, columns in enumerate(S_H_171116_df):
    print(i, columns)

0 id
1 date
2 列1
3 reply
4 lid
5 gid
6 cname
7 label
8 body
9 Epistemic(H)
10 Epistemic(S)
11 Epistemic(FA)
12 ?(2)
13 Coordination(H)
14 Coordination(S)
15 Coordination(FA)
16 ?(2-2)
17 Argumentation(H)
18 Argumentation(S)
19 Argumentation(FA)
20 ?(3)
21 Social(H)
22 Social(S)
23 Social(FA)
24 ?(5)
25 Social Target
26 ?(ST)
27 Unnamed: 27
28 Unnamed: 28
29 Unnamed: 29


In [51]:
for i, columns in enumerate(S_2015_df):
    print(i, columns)

0 発言番号
1 返信元
2 発言の起点
3 起点からの数
4 時間
5 講義番号
6 グループ番号
7 発言者ID
8 ニックネーム
9 宛先者ID
10 宛先
11 発言内容


## 必要なデータ
- id
    - A_W_YYMMDDのNo.(0列目)
    - SH_171116のid(0列目)
    - 斉藤2015の発言番号(0列目)
- reply
    - A_W_YYMMDDの返信元(1列目)
    - SH_171116のreply(3列目)
    - 斉藤2015の返信元(1列目)
- group
    - A_W_YYMMDDのgid(6列目)
    - SH_171116のgid(5列目)
    - 斉藤2015のグループ番号(6列目)
- who
    - A_W_YYMMDDのニックネーム(8列目)
    - SH_171116のcname(6列目)
    - 斉藤2015のニックネーム(8列目)
- body
    - A_W_YYMMDDの発言内容(11列目)
    - SH_171116のbody(8列目)
    - 斉藤2015の発言内容(11列目)
- epistemic_a
    - A_W_YYMMDDのEpistemic(A)(12列目)
    - SH_171116のEpistemic(H)(9列目)
- epistemic_b
    - A_W_YYMMDDのWatanabe(13列目)
    - SH_171116のEpistemic(S)(10列目)
    
    
## 各dfで必要な列
- A_W_YYMMDD[0, 1, 6, 8, 11, 12, 13]
- SH_171116[0, 3, 5, 6, 8, 9, 10]
- 斉藤2015[0, 1, 6, 8, 11]

In [52]:
A_W_YYMMDD_use = [0, 1, 6, 8, 11, 12, 13]
S_H_171116_use = [0, 3, 5, 6, 8, 9, 10]
S_2015_use = [0, 1, 6, 8, 11]

## 使用しない列の削除

In [53]:
# A_W_YYMMDDの列名の統合
A_W_171116_df.columns = A_W_171128_df.columns

delete_columns_list = []
for i, column in enumerate(A_W_171116_df):
    if not i in A_W_YYMMDD_use:
        delete_columns_list.append(column)
        
A_W_171116_df = A_W_171116_df.drop(delete_columns_list, axis=1)
A_W_171128_df = A_W_171128_df.drop(delete_columns_list, axis=1)

delete_columns_list = []
for i, column in enumerate(S_H_171116_df):
    if not i in S_H_171116_use:
        delete_columns_list.append(column)
S_H_171116_df = S_H_171116_df.drop(delete_columns_list, axis=1)

delete_columns_list = []
for i, column in enumerate(S_2015_df):
    if not i in S_2015_use:
        delete_columns_list.append(column)
S_2015_df = S_2015_df.drop(delete_columns_list, axis=1)

In [54]:
A_W_171116_df.head(3)

No. 返信元  gid ニックネーム                          発言内容 Epistemic(A)  Watanabe
0    1  \N    1   まこぴす                   よろしくお願いします！     Off task  Off Task
1   31  \N    1      哲                    よろしくお願いします     Off task  Off Task
2   70  \N    1     仙波  名前なのが恥ずかしいです…\n\nよろしくお願いします！     Off task  Off Task

In [55]:
A_W_171128_df.head(3)

No. 返信元  gid ニックネーム                          発言内容 Epistemic(A)  Watanabe
0   1.0  \N  1.0   まこぴす                   よろしくお願いします！     Off task  Off Task
1  31.0  \N  1.0      哲                    よろしくお願いします     Off task  Off Task
2  70.0  \N  1.0     仙波  名前なのが恥ずかしいです…\n\nよろしくお願いします！     Off task  Off Task

In [56]:
S_H_171116_df.head(3)

id  reply  gid   cname         body Epistemic(H) Epistemic(S)
0  1593      2    2  世界のわたべ  よろしくお願いします。     Off Task     Off Task
1  1598      2    3    ざきさん       よろしくです     Off Task     Off Task
2  1606      2   56       あ       よろしくです     Off Task     Off Task

In [57]:
S_2015_df.head(3)

発言番号 返信元  グループ番号  ニックネーム         発言内容
0  1593  \N       1  世界のわたべ  よろしくお願いします。
1  1598  \N       1    ざきさん       よろしくです
2  1606  \N       1       あ       よろしくです

## データの一致の確認

In [58]:
print(not False in S_H_171116_df['id'].values == S_2015_df['発言番号'].values)
print(not False in S_H_171116_df['cname'].values == S_2015_df['ニックネーム'].values)
print(not False in S_H_171116_df['body'].values == S_2015_df['発言内容'].values)

True
True
True


## 欠損データの置換

In [59]:
S_H_171116_df["reply"] = S_2015_df['返信元']
S_H_171116_df["gid"] = S_2015_df['グループ番号']

In [60]:
S_H_171116_df.head(3)

id reply  gid   cname         body Epistemic(H) Epistemic(S)
0  1593    \N    1  世界のわたべ  よろしくお願いします。     Off Task     Off Task
1  1598    \N    1    ざきさん       よろしくです     Off Task     Off Task
2  1606    \N    1       あ       よろしくです     Off Task     Off Task

## A_W_YYMMDDの結合

In [61]:
concat_border = 299
A_W_171116_df = A_W_171116_df[0:concat_border]
A_W_171128_df = A_W_171128_df[concat_border:]
A_W_df = pd.concat([A_W_171116_df, A_W_171128_df])

In [62]:
A_W_df[295:305]

No. 返信元   gid    ニックネーム  \
295   816.0  \N  11.0  100g188円   
296   876.0  \N  11.0  100g188円   
297  1083.0  \N  11.0      Loki   
298  1139.0  \N  11.0  100g188円   
299  1277.0  \N  11.0  100g188円   
300  1310.0  \N  11.0  100g188円   
301  1324.0  \N  11.0      Loki   
302  1370.0  \N  11.0      Loki   
303  1393.0  \N  11.0  100g188円   
304    24.0  \N  12.0      ざきさん   

                                                発言内容 Epistemic(A)  Watanabe  
295                                  いま見てみたら割ときてましたｗ      On task   On Task  
296                         ムードルだとこないのコンテンツ工程くらいでしたか      On task   On Task  
297  とりあえず共有テキストに書いてみた\n\nスマートフォンの場合が使ったことないから分からない…      On task   On Task  
298                          スマフォではうちも使ったことないですねぇ・・・      On task   On Task  
299                     PCつかえてればスマフォで使おうとはおもあないですよね～      On task   On Task  
300                                               思わ     Off task   On Task  
301                           とりあえず今書いてあることを回答に書きますか     Off task  Off Task  
302                              先に書いてもらいありがとうございます～     Off task  Off Task  
303                                           (＾　＾)＞     Off task  Off Task  
304                                       よろしくお願いします     Off task  Off Task

In [63]:
print(A_W_df.shape)
print(S_H_171116_df.shape)

(4945, 7)
(5017, 7)


## 使用できないレベルの欠損値のある行を取り除く

In [64]:
# 要素含まれるNanの数を確認
A_W_df.isnull().sum()

No.             3
返信元             3
gid             3
ニックネーム          3
発言内容            3
Epistemic(A)    3
Watanabe        3
dtype: int64

In [65]:
# A_W_dfの人間が付けたラベル列のNanを取り除く
A_W_df = A_W_df.dropna(subset=['Epistemic(A)', 'Watanabe'])
# 要素にNanが含まれる列を再確認
A_W_df.isnull().any()

No.             False
返信元             False
gid             False
ニックネーム          False
発言内容            False
Epistemic(A)    False
Watanabe        False
dtype: bool

In [66]:
# 要素に含まれるNanの数を確認
S_H_171116_df.isnull().sum()

id              0
reply           0
gid             0
cname           0
body            0
Epistemic(H)    1
Epistemic(S)    4
dtype: int64

In [67]:
# S_H_171116_dfの人間が付けたラベル列のNanを取り除く
S_H_171116_df = S_H_171116_df.dropna(subset=['Epistemic(H)', 'Epistemic(S)'])
# 要素にNanが含まれる列を再確認
S_H_171116_df.isnull().any()

id              False
reply           False
gid             False
cname           False
body            False
Epistemic(H)    False
Epistemic(S)    False
dtype: bool

## 重複投稿の排除
- 発言者と発言内容が重複している行を排除する
- これだと連投だけでなく、「よろしくお願いします」などの定型的な発言がスキップされてしまう
- 他の人の投稿を挟んでの連投を排除できる
- 「よろしくお願いいたします」などの定型文はoff task
- 文の前後関係においてもノイズになりやすそう(よろしくお願いしますから発生する次の発言は多様すぎて関連性が薄いと思いたい)
- **元データのラベル付けルールの整合性とかがかなり怪しいから荒っぽい前処理くらい許して欲しい**

In [68]:
A_W_df = A_W_df.drop_duplicates(subset=['ニックネーム', '発言内容'])
S_H_171116_df = S_H_171116_df.drop_duplicates(subset=['cname', 'body'])

## ラベルの表記ゆれの修正
ラベルを小文字に統一する

### Epistemicの修正

In [69]:
# 表記ゆれの修正と確認
A_W_df['Epistemic(A)'] = A_W_df['Epistemic(A)'].str.lower()
counter = Counter(A_W_df['Epistemic(A)'])
print(counter)
A_W_df['Watanabe'] = A_W_df['Watanabe'].str.lower()
counter = Counter(A_W_df['Watanabe'])
print(counter)

Counter({'off task': 2392, 'on task': 2092, 'no sense': 119})
Counter({'off task': 2444, 'on task': 2148, 'no sense': 11})


In [70]:
# 表記ゆれの確認
S_H_171116_df['Epistemic(S)'] = S_H_171116_df['Epistemic(S)'].str.lower()
counter = Counter(S_H_171116_df['Epistemic(S)'])
print(counter)
S_H_171116_df['Epistemic(H)'] = S_H_171116_df['Epistemic(H)'].str.lower()
counter = Counter(S_H_171116_df['Epistemic(H)'])
print(counter)

Counter({'off task': 2545, 'on task': 2179, 'no sense': 10})
Counter({'off task': 2560, 'on task': 2161, 'no sense': 13})


## No Senseの排除

In [71]:
A_W_df = A_W_df[A_W_df['Epistemic(A)'] != 'no sense']
A_W_df = A_W_df[A_W_df['Watanabe'] != 'no sense']
print(Counter(A_W_df['Epistemic(A)']))
print(Counter(A_W_df['Watanabe']))

Counter({'off task': 2390, 'on task': 2091})
Counter({'off task': 2337, 'on task': 2144})


In [72]:
S_H_171116_df = S_H_171116_df[S_H_171116_df['Epistemic(H)'] != 'no sense']
S_H_171116_df = S_H_171116_df[S_H_171116_df['Epistemic(S)'] != 'no sense']
print(Counter(S_H_171116_df['Epistemic(H)']))
print(Counter(S_H_171116_df['Epistemic(S)']))

Counter({'off task': 2558, 'on task': 2158})
Counter({'off task': 2538, 'on task': 2178})


## 相談なしで一致しているラベルの抽出

### 一致しているEpistemic

In [73]:
A_W_match_df = A_W_df[A_W_df['Epistemic(A)'] == A_W_df['Watanabe']].reset_index(drop=True)
A_W_match_df.head()

No. 返信元  gid ニックネーム                          発言内容 Epistemic(A)  Watanabe
0    1.0  \N  1.0   まこぴす                   よろしくお願いします！     off task  off task
1   31.0  \N  1.0      哲                    よろしくお願いします     off task  off task
2   70.0  \N  1.0     仙波  名前なのが恥ずかしいです…\n\nよろしくお願いします！     off task  off task
3  119.0  \N  1.0   まこぴす                早速課題やっちゃいましょう！     off task  off task
4  163.0  \N  1.0     仙波     やっちゃいましょう\n\nmoodleはゴミです！      on task   on task

In [74]:
S_H_match_df = S_H_171116_df[S_H_171116_df['Epistemic(H)'] == S_H_171116_df[
    'Epistemic(S)']].reset_index(drop=True)
S_H_match_df.head()

id reply  gid   cname                body Epistemic(H) Epistemic(S)
0  1593    \N    1  世界のわたべ         よろしくお願いします。     off task     off task
1  1598    \N    1    ざきさん              よろしくです     off task     off task
2  1606    \N    1       あ              よろしくです     off task     off task
3  1659    \N    1    ざきさん  みなさんファイルをアップしましたか？     off task     off task
4  1683    \N    1       あ           今アップしました！     off task     off task

In [75]:
A_W_match_df.isnull().sum()

No.             0
返信元             0
gid             0
ニックネーム          0
発言内容            0
Epistemic(A)    0
Watanabe        0
dtype: int64

In [76]:
S_H_match_df.isnull().sum()

id              0
reply           0
gid             0
cname           0
body            0
Epistemic(H)    0
Epistemic(S)    0
dtype: int64

## A_WとS_Hのheaderの統一

In [77]:
header = ['id', 'reply', 'group_id', 'cname', 'body', 'Epistemic_A', 'Epistemic_B']
A_W_df.columns = header
S_H_171116_df.columns = header
A_W_match_df.columns = header
S_H_match_df.columns = header

## A_WとS_Hの結合

In [78]:
All_df = pd.concat([A_W_df, S_H_171116_df], ignore_index=True)
print(All_df.shape)
print(Counter(All_df['Epistemic_A']))
print(Counter(All_df['Epistemic_B']))

(9197, 7)
Counter({'off task': 4948, 'on task': 4249})
Counter({'off task': 4875, 'on task': 4322})


## 相談なしで一致したA_W_match_dfとS_H_match_dfの結合

In [81]:
match_df = pd.concat([A_W_match_df, S_H_match_df], ignore_index=True)
print(match_df.shape)
print(Counter(match_df['Epistemic_A']))
print(Counter(match_df['Epistemic_B']))

(8460, 7)
Counter({'off task': 4543, 'on task': 3917})
Counter({'off task': 4543, 'on task': 3917})


In [83]:
print(match_df.shape[0]/All_df.shape[0] *100)
print(len(match_df[match_df["Epistemic_A"] == "off task"]))
print(len(match_df[match_df["Epistemic_A"] == "on task"]))

91.98651734261172
4543
3917


## 一応データの整理はここまで
- 必要な前処理などは気づき次第、適時xq修正する

In [84]:
All_df.head(5)

id reply  group_id cname                          body Epistemic_A  \
0    1.0    \N       1.0  まこぴす                   よろしくお願いします！    off task   
1   31.0    \N       1.0     哲                    よろしくお願いします    off task   
2   70.0    \N       1.0    仙波  名前なのが恥ずかしいです…\n\nよろしくお願いします！    off task   
3  119.0    \N       1.0  まこぴす                早速課題やっちゃいましょう！    off task   
4  163.0    \N       1.0    仙波     やっちゃいましょう\n\nmoodleはゴミです！     on task   

  Epistemic_B  
0    off task  
1    off task  
2    off task  
3    off task  
4     on task

In [85]:
match_df.head(5)

id reply  group_id cname                          body Epistemic_A  \
0    1.0    \N       1.0  まこぴす                   よろしくお願いします！    off task   
1   31.0    \N       1.0     哲                    よろしくお願いします    off task   
2   70.0    \N       1.0    仙波  名前なのが恥ずかしいです…\n\nよろしくお願いします！    off task   
3  119.0    \N       1.0  まこぴす                早速課題やっちゃいましょう！    off task   
4  163.0    \N       1.0    仙波     やっちゃいましょう\n\nmoodleはゴミです！     on task   

  Epistemic_B  
0    off task  
1    off task  
2    off task  
3    off task  
4     on task

# 現状の整理
- 人間A, Bそれぞれのタグ取り出しはできそう
- 人間ABの相談なし一致のタグはA,Bのタグを比較すればできそう    

## ここからは分類器に入力するための整形処理
- Epistemicの分類
    - 人間Aのタグ
    - 人間Bのタグ
    - 人間ABの相談なし一致のタグ
- Argumentationの分類
    - 人間Aのタグ
    - 人間Bのタグ
    - 人間ABの相談なし一致のタグ

## どのように処理していくか
- これまでは、扱うラベルなどが変わるたびにnumpy配列の列番号を変えなくてはならなかった
- これからは、扱うデータの形式を統一したい
- そのためにどうするか
    - 必要なデータの構成を決める
    - data frameから必要のない列を削ったnumpy配列を作成する

## 分類器に入力するために必要なデータ
- id
- reply
- group_id
- cname
- body
- label(Epistemic or Argumentation)

In [86]:
# データの形式の定義
data_format = ['id', 'reply', 'group_id', 'cname', 'body', 'label']
# 名前からデータのindexに変換する辞書
data_index = {column : i for i, column in enumerate(data_format)}
print(data_index)

{'id': 0, 'reply': 1, 'group_id': 2, 'cname': 3, 'body': 4, 'label': 5}


In [87]:
id = 0
reply = 1
group_id = 2
cname = 3
body = 4
label = 5

df_index = {column: i for i, column in enumerate(All_df)}
print(df_index)

{'id': 0, 'reply': 1, 'group_id': 2, 'cname': 3, 'body': 4, 'Epistemic_A': 5, 'Epistemic_B': 6}


### All_dfから共通して取り出す列名
- ['id', 'reply', 'group_id', 'cname', 'body']

### 変動する列名
- label

In [88]:
df_use = [0, 1, 2, 3, 4]
# All_df_index['ラベル名']を変更して取り出すラベルを分ける
# label_position = df_index['Epistemic_A']
# label_position = df_index['Epistemic_B']
label_position = df_index['Epistemic_B']
df_use.append(label_position)
print(df_use)

[0, 1, 2, 3, 4, 6]


## 必要のない列を取り除いて、入力用のdfを作成

In [89]:
delete_columns_list = []
# for i, column in enumerate(All_df):
for i, column in enumerate(match_df):
    if not i in df_use:
        delete_columns_list.append(column)
# input_df = All_df.drop(delete_columns_list, axis=1)
input_df = match_df.drop(delete_columns_list, axis=1)
# input_df = A_W_consulted_df.drop(delete_columns_list, axis=1)
input_df.columns = data_format

In [90]:
input_df

id reply  group_id    cname  \
0        1.0    \N       1.0     まこぴす   
1       31.0    \N       1.0        哲   
2       70.0    \N       1.0       仙波   
3      119.0    \N       1.0     まこぴす   
4      163.0    \N       1.0       仙波   
5      194.0    \N       1.0     まこぴす   
6      302.0    \N       1.0        哲   
7      309.0    \N       1.0       仙波   
8      385.0    \N       1.0     まこぴす   
9      500.0    \N       1.0       仙波   
10     528.0    \N       1.0       仙波   
11     693.0    \N       1.0     まこぴす   
12     715.0    \N       1.0       仙波   
13     791.0    \N       1.0       仙波   
14     813.0    \N       1.0     まこぴす   
15     830.0   746       1.0        哲   
16     831.0    \N       1.0       仙波   
17     899.0    \N       1.0     まこぴす   
18     916.0    \N       1.0       仙波   
19     934.0   899       1.0        哲   
20     964.0    \N       1.0     まこぴす   
21     990.0    \N       1.0       仙波   
22    1017.0    \N       1.0       \N   
23    1031.0    \N       1.0     まこぴす   
24    1048.0    \N       1.0       仙波   
25    1066.0    \N       1.0        哲   
26    1101.0    \N       1.0       仙波   
27    1107.0    \N       1.0     まこぴす   
28      27.0    \N       2.0      マニン   
29      90.0    \N       2.0  Houdini   
...      ...   ...       ...      ...   
8430  6189.0  6183   10001.0      みずき   
8431  6204.0  6189   10001.0      まささ   
8432  6224.0    \N   10001.0     オノヅカ   
8433  6237.0  6224   10001.0      みずき   
8434  6241.0  6224   10001.0      まささ   
8435  6243.0    \N   10001.0     オノヅカ   
8436  6254.0    \N   10001.0     オノヅカ   
8437  6257.0  6254   10001.0      まささ   
8438  6264.0  6254   10001.0      みずき   
8439  6266.0    \N   10001.0     オノヅカ   
8440  6278.0  6266   10001.0      まささ   
8441  6280.0  6266   10001.0      みずき   
8442  6288.0    \N   10001.0     オノヅカ   
8443  6326.0    \N   10001.0     オノヅカ   
8444  6347.0    \N   10001.0      みずき   
8445  6363.0    \N   10001.0      まささ   
8446  6388.0    \N   10001.0     オノヅカ   
8447  6404.0  6388   10001.0      みずき   
8448  6410.0  6404   10001.0     オノヅカ   
8449  6415.0    \N   10001.0      まささ   
8450  6419.0  6404   10001.0      まささ   
8451  6435.0  6415   10001.0      みずき   
8452  6449.0    \N   10001.0     オノヅカ   
8453  6451.0  6443   10001.0      みずき   
8454  6459.0    \N   10001.0      みずき   
8455  6463.0  6459   10001.0      まささ   
8456  6467.0  6459   10001.0     オノヅカ   
8457  6477.0    \N   10001.0     オノヅカ   
8458  6491.0  6477   10001.0      まささ   
8459  6521.0    \N   10001.0     オノヅカ   

                                                   body     label  
0                                           よろしくお願いします！  off task  
1                                            よろしくお願いします  off task  
2                          名前なのが恥ずかしいです…\n\nよろしくお願いします！  off task  
3                                        早速課題やっちゃいましょう！  off task  
4                             やっちゃいましょう\n\nmoodleはゴミです！   on task  
5                                            使いにくいです(笑)   on task  
6                                                  同意です   on task  
7     以前インタラクティブアート受講していたのですが、その時に課題が不具合で出せなくなっていた時期...   on task  
8     学習環境として必要最低限の機能は備えていると思うが、操作性の面ではPCの最低限の知識があるこ...   on task  
9                                              ほぼほぼ同意です   on task  
10                                    他に何か考えてることなどありますか   on task  
11                                          他に意見ありますかー？   on task  
12                                              自分はないです   on task  
13                  まこびすさんがいい感じにまとめてくれたので自分はこれでいいと思います！   on task  
14                             １度共有テキスト全部消して現在の回答案書きます！  off task  
15                       > 哲さんどうですか？\n\n自分もみなさんの意見に同意です   on task  
16                                                 了解です   on task  
17                      こちらの回答案でよければ課題回答しちゃいますけど大丈夫ですか？  off task  
18                                              自分は平気です  off task  
19                                          よろしくお願いします！  off task  
20                                     

# 出現頻度が少ないラベルを確認

In [91]:
counter = Counter(input_df['label'])
print(counter)

Counter({'off task': 4543, 'on task': 3917})


In [92]:
input_data = input_df.values

In [93]:
def preprocess(xs, trim_citation=False):
    zs = [""] * len(xs)
    '''全角空白'''
    for i, l in enumerate(xs):
        xs[i] = re.sub('　', ' ', str(l))
    '''多重引用'''
    for i, l in enumerate(xs):
        xs[i] = re.sub('＆ｇｔ；', '＞', l)
    '''引用部分は除外'''
    count_refer = 0
    for i, l in enumerate(xs):
        sen = ""
        have_refer = 1
        for x in l.split('\n'):
            if len(x) == 0:
                continue
            elif x[0].encode('utf-8') == '＞'.encode('utf-8'):
                have_refer = 1
                x = x[1:]
                if len(x) > 0 and x[0].encode('utf-8') != '＞'.encode('utf-8') and \
                        x[0:2].encode('utf-8') != ' ＞'.encode('utf-8') and x != ' ':
                    if trim_citation:
                        zs[i] += x + '\n'
                    else:
                        sen = sen + "＞ " + x + '\n'
                continue
            else:
                sen = sen + x + '\n'
        xs[i] = sen.strip()
        count_refer += have_refer
    '''顔文字'''
    pat = re.compile('（[^ぁ-んァ-ン一-龠]+?）', re.U)
    for i, l in enumerate(xs):
        xs[i] = re.sub(pat, ' KAOMOJI ', l)
    '''日本語及び記号{？！ー〜。、} のみ有効'''
    pat = re.compile('[^＞ Ａ-Ｚａ-ｚ０-９ぁ-んァ-ン一-龠？！ー〜。、\n]+?', re.U)
    for i, l in enumerate(xs):
        xs[i] = re.sub(pat, '', l)
    '''記号繰り返し'''
    pat_1 = re.compile('[ｗ]{1,}', re.U)
    pat_2 = re.compile('[？]{1,}', re.U)
    pat_3 = re.compile('[！]{1,}', re.U)
    pat_4 = re.compile('[～]{1,}', re.U)
    pat_5 = re.compile('[ー]{1,}', re.U)
    pat_6 = re.compile('[。]{1,}', re.U)

    for i, l in enumerate(xs):
        l = re.sub(pat_1, 'ｗ', l)
        l = re.sub(pat_2, '？', l)
        l = re.sub(pat_3, '！', l)
        l = re.sub(pat_4, '～', l)
        l = re.sub(pat_5, 'ー', l)
        l = re.sub(pat_6, '。', l)
        xs[i] = l

    if trim_citation:
        return zs
    else:
        return None

In [94]:
def make_xs(orig, trim_citation=False, use_seq2seq=False):

    tmp = np.copy(orig)
    xs = tmp[:, body]
    zs = preprocess(xs, trim_citation=(trim_citation or use_seq2seq))

    if use_seq2seq:
        tmp_prev = np.roll(tmp, 1, axis=0)
        tmp_prev[0, 0] = ""
        ts = tmp[:, [1, 2]]
        xs_prev = tmp_prev[:, body]

        for i in range(len(tmp)):
            if tmp[i, group_id] != tmp_prev[i, group_id]:  # group が異なる
                xs_prev[i] = ""
        preprocess(xs_prev, trim_citation=True)

        for i in range(len(xs)):
            replay_to = tmp[i, 2]
            if zs[i] != "":
                xs_prev[i] = zs[i]
            elif replay_to != -1:
                a = xs[:i][tmp[:i, 1] == replay_to]
                if len(a) != 0:
                    xs_prev[i] = a[-1]

        return xs, xs_prev
    else:
        return xs

In [95]:
def get_doc(xs):
    doc = []
    for ll in xs:
        sen = []
        for l in ll.split('\n'):
            for w in mc.parse(l).split('\n'):
                mx = w.split('\t')
                if len(mx[0]) > 0:
                    sen.append(mx[0])
        doc.append(sen)
    return doc

In [96]:
xs, xs_pre = make_xs(input_data, trim_citation=True, use_seq2seq=True)
doc = get_doc(xs)
doc_pre = get_doc(xs_pre)

In [97]:
for i in doc[-10:]:
    print(" ".join(i))

今 出 て き た 意見 全部 繋げ ちゃう ？ EOS 今 つなげよ う と し て ます ！ EOS ありがとう ござい ます ！ EOS
次世代 として 発表 さ れ た 規格 で 、 正式 名称 は 略称 、 旧称 。 EOS 広範 な デバイス を カバー する とともに 、 ビッグ データ 活用 に 向け た の 仕様 を 規定 し て いる 。 EOS こういう 説明 も あり まし た 。 EOS なんか こっち の 方 が かっこいい EOS
提出 お願い し ます EOS
共有 テキスト ので 良い と 思い ます EOS 私 も そう 思い ます 、 分かり やすく て いい です ！ EOS じゃあ この まま 提出 し ます ！ EOS
課題 提出 完了 です ！ EOS
課題 提出 完了 です ！ EOS ありがとう ござい ます ！ EOS
課題 提出 完了 です ！ EOS ありがとう ござい ます EOS
課題 やり ます か EOS
課題 やり ます か EOS やり ましょ う 、 共有 テキスト いったん 消し ちゃい ます ね EOS
何 を の 部分 の 選択肢 が 後 つ くらい 欲しい か な EOS


# 単語の数値変換
wd_setにdocとdoc_preに格納されている単語(listの要素単位)をカウンターで数え上げる  
wd_aryはカウントされた単語のリスト(Counter==dictのkey)  
wd_cntは各単語の出現数(dictのvalue)  
wd_aryを単語の出現数(昇順)に操作  
wd_cntを昇順に直す出現数で昇順に並べ替え済みのwd_aryをもとにenumrateで単語:単語idのdictを作成

python2→3の変更点で、dict.keys()がiter的なので返ってきてしまうからlist()でキャストする必要あった

In [98]:
wd_set = Counter([x for sen in doc + doc_pre for x in sen])
wd_ary = np.array(list(wd_set.keys()))
wd_cnt = np.array(list(wd_set.values()))
wd_ary = wd_ary[np.argsort(wd_cnt)[::-1]]
wd_cnt.sort()
wd_cnt = wd_cnt[::-1]
wd_to_id = {wd: i for i, wd in enumerate(wd_ary)}
lb_to_id = {lb: i for i, lb in enumerate({x for x in input_data[:, label]})}
id_to_wd = {wd_to_id[wd]: wd for wd in wd_to_id.keys()}
id_to_lb = {lb_to_id[lb]: lb for lb in lb_to_id.keys()}

In [99]:
for i in range(10):
    print(list(wd_ary)[i], list(wd_cnt)[i])
    #print(list(wd_set.keys())[i], list(wd_set.values())[i])

EOS 26180
の 10331
て 7217
です 7071
か 6838
が 6751
に 6739
ます 6646
、 6511
は 6484


In [100]:
KIND = len(list(lb_to_id.keys()))
CUT_OFF = 2

print("words kinds:",
      len(wd_cnt), "words>=" + str(CUT_OFF) + ":", np.sum(wd_cnt >= CUT_OFF))
print("all words num:", np.sum(wd_cnt))
print("all words num:", np.sum(wd_cnt[wd_cnt >= CUT_OFF]))

other_id = np.sum(wd_cnt >= CUT_OFF)
wd_to_id.update({wd: other_id for wd in wd_ary[wd_cnt < CUT_OFF]})
print(other_id)
id_to_wd[other_id] = '[X]'

print(max(wd_to_id.values()))

with open('data/Epistemic_FA_id_to_wd.pickle', mode='wb') as f:
    pickle.dump(id_to_wd, f)
with open('data/Epistemic_FA_wd_to_id.pickle', mode='wb') as f:
    pickle.dump(wd_to_id, f)
with open('data/Epistemic_FA_wd_set.pickle', mode='wb') as f:
    pickle.dump(wd_set, f)
with open('data/Epistemic_FA_id_to_lb.pickle', mode='wb') as f:
    pickle.dump(id_to_lb, f)
pd.DataFrame(
    np.array(list(id_to_lb.items())).T, index=["ラベルid", "ラベルname"])

words kinds: 5411 words>=2: 5315
all words num: 312094
all words num: 311998
5315
5315


0        1
ラベルid           0        1
ラベルname  off task  on task

In [101]:
of = open("data/Epistemic_FA_edu_data.txt", "w")
for i, (sen, lb) in enumerate(zip(np.array(doc), input_data[:, label])):
    sen_str = "".join([str(wd_to_id[wd]) + " " for wd in sen])
    print_str = str(lb_to_id[lb]) + " " + str(lb_to_id[lb]) + " " + sen_str
    print(print_str, file=of)
of.close()

of = open("data/Epistemic_FA_edu_data_pre.txt", "w")
for i, sen in enumerate(np.array(doc_pre)):
    sen_str = "".join([str(wd_to_id[wd]) + " " for wd in sen])
    print(sen_str, file=of)
of.close()

print(max([len(sen) for sen in doc]))

print(KIND)

313
2


# train